### Gerekli Kütüphaneler


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Veriyi Oku ve Çıktı Ver

In [11]:
df = pd.read_excel(r"C:\Users\ASUS\Desktop\Bitirme\1-data_cleaning\birlesik_dosya_(0-1000).xlsx")
df.head(5)

,price,m2,age,floor,district,neighbor,total_room
0,2229999\n TL,110,0.0,1,Beylikdüzü,Marmara Mah.,3.0
1,2650000\n TL,90,0.0,4,Esenyurt,Güzelyurt Mah.,2.0
2,3449999\n TL,130,0.0,2,Beylikdüzü,Adnan Kahveci Mah.,3.0
3,5475000\n TL,125,21.0,4,Pendik,Harmandere Mah.,3.0
4,3450980\n TL,110,0.0,1,Beylikdüzü,Adnan Kahveci Mah.,2.0


# Veri Düzenleme

Veri Kümesinin Genel Özeti

Sayısal Değişkenlerin Tanımlayıcı İstatistikleri

Eksik Değer Analizi

Kategorik Değişkenlerin Dağılımı

Aykırı Değer Kontrolü (Outlier Detection)

Korelasyon Analizi

Veri Temizleme (Tutarsızlık ve Tekrar Eden Değerler)

Hedef ve Özellik İlişkileri (Varsa Hedef Değişken)

Yinelenen (Duplicate) Satır Kontrolü

Veri Tiplerinin Kontrolü ve Dönüşümler


### Genel Bakış

In [15]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18481 entries, 0 to 18480
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       18481 non-null  object 
 1   m2          18481 non-null  int64  
 2   age         18480 non-null  float64
 3   floor       18481 non-null  object 
 4   district    18481 non-null  object 
 5   neighbor    18481 non-null  object 
 6   total_room  18427 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1010.8+ KB


(18481, 7)

In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
m2,18481.0,123.599751,48.560078,50.0,90.0,110.0,145.0,350.0
age,18480.0,13.184037,14.483101,0.0,1.0,8.0,25.0,340.0
total_room,18427.0,3.650024,1.860225,2.0,3.0,3.0,4.0,165.0


### NA Değer Kontrolü

In [20]:
df.isna().sum().sort_values(ascending=False)

total_room    54
age            1
price          0
m2             0
floor          0
district       0
neighbor       0
dtype: int64

### Kopya Veriler

Web Scrapping Kaynaklı

Kopya (tamamen aynı olan) verilerin sayısı: 1586

Yaklaşık %8.6'sı tekrar eden veri.

In [23]:
df.duplicated().sum()

np.int64(1563)

In [25]:
df = df.drop_duplicates()
df.shape

(16918, 7)



### 1. "price"

In [28]:
print(repr(df["price"].iloc[0])) #Hücrenin tam çıktısı

'2229999\n                  TL'


In [30]:
df["price"] = df["price"].str.replace(r"\n                  TL", "", regex=True)
df.head(5)

,price,m2,age,floor,district,neighbor,total_room
0,2229999,110,0.0,1,Beylikdüzü,Marmara Mah.,3.0
1,2650000,90,0.0,4,Esenyurt,Güzelyurt Mah.,2.0
2,3449999,130,0.0,2,Beylikdüzü,Adnan Kahveci Mah.,3.0
3,5475000,125,21.0,4,Pendik,Harmandere Mah.,3.0
4,3450980,110,0.0,1,Beylikdüzü,Adnan Kahveci Mah.,2.0


In [32]:
print(df["price"].dtype)
df["price"].isna().sum()

object


np.int64(0)

In [34]:
df["price"] = df["price"].astype(int)
print(df["price"].dtype)

int64


**"price" kolonu tamamlandı.**

### m2


In [37]:
print(df["m2"].dtype)
df["m2"].isna().sum()

int64


np.int64(0)

##### m2 için bir sorun gözükmüyor

### "age"

In [41]:
print(df["age"].dtype)
df["age"].isna().sum()

float64


np.int64(1)

In [43]:
df.dropna(subset=["age"], inplace=True)
df["age"] = df["age"].astype(int)
print(df["age"].dtype)
df["age"].isna().sum()

int64


np.int64(0)

##### Bir sorun yok

### "floor"

In [47]:
print(df["floor"].dtype)
df["floor"].isna().sum()

object


np.int64(0)

In [49]:
df["floor"].unique()

array([1, 4, 2, 3, 'Ara Kat', 5, 8, 'Yüksek Giriş', 'Giriş Katı', 6,
       'Bahçe Katı', 7, 10, 'Villa Katı', 'En Üst Kat', 13, 12,
       'Yarı Bodrum', 15, 'Zemin', 'Çatı Katı', 11, 18, '21 ve üzeri', 9,
       20, 'Teras Katı', 'Kot 1', 'Kot 2', 17, 14, 'Kot 3', 'Bodrum',
       'Bodrum ve Zemin', 19, 16], dtype=object)

In [51]:
kat_haritasi = {
    "Ara Kat": 3,
    "Yüksek Giriş": 0.5,
    "Giriş Katı": 0,
    "Bahçe Katı": 0,
    "Villa Katı": 0,
    "En Üst Kat": 12,
    "Yarı Bodrum": -0.5,
    "Zemin": 0,
    "Çatı Katı": 12,
    "21 ve üzeri": 24,
    "Teras Katı": 9,
    "Kot 1": -1,
    "Kot 2": -2,
    "Kot 3": -3,
    "Bodrum": -1,
    "Bodrum ve Zemin": -0.5
}

df["floor"] = df["floor"].replace(kat_haritasi)
df["floor"] = pd.to_numeric(df["floor"], errors="coerce") # Hatalı değerleri nan yapar

df["floor"].isna().sum()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16068\2144047913.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["floor"] = df["floor"].replace(kat_haritasi)


np.int64(0)

In [53]:
print(df["floor"].unique())
print(df["floor"].dtype)

[ 1.   4.   2.   3.   5.   8.   0.5  0.   6.   7.  10.  12.  13.  -0.5
 15.  11.  18.  24.   9.  20.  -1.  -2.  17.  14.  -3.  19.  16. ]
float64


### total_room

In [56]:
print(df["total_room"].dtype)
df["total_room"].isna().sum()

float64


np.int64(48)

In [58]:
df[df["total_room"].isna()].head()

,price,m2,age,floor,district,neighbor,total_room
296,1325000,55,13,3.0,Esenyurt,Zafer Mah.,NaN
390,3580000,95,12,3.0,Esenyurt,Turgut Özal Mah.,NaN
757,4499000,70,15,3.0,Kağıthane,Sultan Selim Mah.,NaN
1004,1990000,55,6,12.0,Esenyurt,Zafer Mah.,NaN
1309,4100000,70,13,3.0,Ataşehir,Esatpaşa Mah.,NaN


In [60]:
df["total_room"] = df.groupby("neighbor")["total_room"].transform(
    lambda x: x.fillna(x.mean()))
df["total_room"].isna().sum()

np.int64(0)

In [62]:
df["total_room"] = df["total_room"].round().astype(int)
print(df["total_room"].dtype)

int64


### neighbor ve district

Encoding işleminden önce süreçte bir sorun çıkmaması için:

In [65]:
df["neighbor"] = df["neighbor"].str.strip()
df["district"] = df["district"].str.strip()

# Sonuç

Veriler tipleri kabaca ML sistemine hazır oldu

district ve neighbor kolonları için kategorik verilerden sayısal değerlere encoding işlmi yapılacak

güncel veriyi çıkarmak için:

In [68]:
df.to_csv("clean_data.csv", index=False)
print("clean_data.csv kaydedildi!")

clean_data.csv kaydedildi!
